In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [ ]:
import numpy as np
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Flatten, Activation
from keras.layers import Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.optimizers import SGD
from sklearn.metrics import precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Задаем seed для повторяемости результатов
np.random.seed(42)

In [ ]:
# Загружаем данные
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [ ]:
# Размер мини-выборки
batch_size = 32
# Количество классов изображений
nb_classes = 10
# Количество эпох для обучения
nb_epoch = 25
# Размер изображений
img_rows, img_cols = 32, 32
# Количество каналов в изображении: RGB
img_channels = 3

In [ ]:
# Нормализуем данные
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

# Преобразуем метки в категории
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

In [ ]:
# Создаем последовательную модель
model = Sequential()

# Первый сверточный слой
model.add(Conv2D(32, (3, 3), padding='same', input_shape=(32, 32, 3), activation='relu'))

# Второй сверточный слой
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))

# Первый слой подвыборки
model.add(MaxPooling2D(pool_size=(2, 2)))

# Слой регуляризации Dropout
model.add(Dropout(0.25))

# Третий сверточный слой
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))

# Четвертый сверточный слой
model.add(Conv2D(64, (3, 3), activation='relu'))

# Второй слой подвыборки
model.add(MaxPooling2D(pool_size=(2, 2)))

# Слой регуляризации Dropout
model.add(Dropout(0.25))

# Слой преобразования данных из 2D представления в плоское
model.add(Flatten())

# Полносвязный слой для классификации
model.add(Dense(512, activation='relu'))

# Слой регуляризации Dropout
model.add(Dropout(0.5))

# Выходной полносвязный слой
model.add(Dense(nb_classes, activation='softmax'))


In [ ]:
print(model.summary())      # ნეირონული ქსელის სტრუქტურის გამტანა კონსოლში

In [ ]:
# Задаем параметры оптимизации
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


In [ ]:
# Обучаем модель
history = model.fit(X_train, Y_train,
                    batch_size=batch_size,
                    epochs=nb_epoch,
                    validation_split=0.1,
                    shuffle=True,
                    verbose=2)

In [ ]:
# Построение графика правильности
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Точность модели')
plt.xlabel('Эпоха')
plt.ylabel('Точность')
plt.legend(['Тренироваться', 'Проверка'], loc='upper left')
plt.show()

In [ ]:
# Оцените обученную модель на тестовом наборе
scores = model.evaluate(X_test, Y_test, verbose=0)
print("Точность на тестовом наборе: %.2f%%" % (scores[1]*100))

In [ ]:
# Делайте прогнозы на тестовом наборе
predictions = model.predict(X_test)
y_pred = np.argmax(predictions, axis=1)

In [ ]:
# Рассчитать точность, отзыв и оценку F1
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

In [ ]:
print("Точность: %.2f" % precision)
print("Отзывать: %.2f" % recall)
print("Оценка F1: %.2f" % f1)

In [ ]:
from keras.preprocessing import image
from PIL import Image

In [ ]:
# Загрузите загруженное изображение и измените его размер на (32, 32)
uploaded_image_path = 'uploaded_image.jpg'
uploaded_image = Image.open(uploaded_image_path)
uploaded_image = uploaded_image.resize((32, 32))
uploaded_image = np.array(uploaded_image)
uploaded_image = np.expand_dims(uploaded_image, axis=0)
uploaded_image = uploaded_image.astype('float32') / 255.0

In [ ]:
# Сделать прогноз
prediction = model.predict(uploaded_image)
predicted_class = np.argmax(prediction)
class_labels = ['Самолет', 'Автомобиль', 'Птица', 'Кот', 'Олень', 'Собака', 'Лягушка', 'Лошадь', 'Корабль', 'Грузовик']
predicted_class_label = class_labels[predicted_class]
print("Прогнозируемый класс: ", predicted_class_label)

In [ ]:
# Отображение загруженного изображения с предсказанной меткой класса
import matplotlib.pyplot as plt

plt.imshow(uploaded_image[0])
plt.title('Прогнозируемый класс: ' + predicted_class_label)
plt.axis('off')
plt.show()